# Hadronic EoS Generation Worker

## Detailed Analysis

This script is responsible for generating synthetic Equation of State (EoS) curves for hadronic neutron stars. It operates as a worker process that creates "Label 0" data for the machine learning pipeline.

The generator uses a "Inverse Sampling" technique. Instead of randomly guessing EoS parameters and hoping to find massive stars, it selects target maximum masses first (e.g., between 2.0 and 3.0 $M_{\odot}$) and mathematically infers the scaling parameters required to generate them. This ensures the dataset is populated with realistic, high-mass neutron stars consistent with modern observations (like PSR J0740+66), minimizing computational waste.

## Physics and Math

The generation process relies on the properties of homologous scaling for the TOV equations.

### Homologous Scaling
If an EoS $P(\epsilon)$ yields a maximum mass $M_{ref}$, a scaled EoS $P'(\epsilon') = \alpha P(\epsilon)$ (where energy density is also scaled) will yield a new maximum mass $M_{new}$. The scaling relationship is:

$$
M_{new} = \frac{M_{ref}}{\sqrt{\alpha}} \quad \Rightarrow \quad \alpha = \left( \frac{M_{ref}}{M_{new}} \right)^2
$$

The script defines the target mass as a perturbation of the reference mass:

$$
M_{new} = M_{ref} (1 + \delta)
$$

where $\delta$ is the scaling perturbation. The scaling factor $\alpha$ is then computed to enforce this mass.

### Causality Constraint
Physical matter cannot transmit information faster than light. The speed of sound squared is defined as:

$$
c_s^2 = \frac{dP}{d\epsilon}
$$

The script enforces the causal limit $c_s^2 \le 1$. If an EoS becomes "stiff" enough to violate this limit at low densities (defined by `CAUSALITY_EPS_LIMIT`), it is rejected as unphysical.

### Slope Feature
To capture the topology of the Mass-Radius curve, the local slope is approximated numerically:

$$
\text{Slope} = \frac{\Delta R}{\Delta M} \approx \frac{R(M+\Delta M) - R(M)}{\Delta M}
$$

This feature distinguishes hadronic branches (typically $dR/dM < 0$) from quark branches (often $dR/dM > 0$).

## Code Walkthrough

### 1. Smart Parameter Selection
The core logic resides in how parameters are chosen to guarantee a valid star.
Two parent models (`nA`, `nB`) are selected from the library, and a mixing weight $w$ is chosen. The "Base Max Mass" is the weighted average of the parents.

```python
# What delta do we need to hit the mass floor?
req_delta_min = (m_min_target / base_max_m) - 1.0

# What delta do we need to hit the mass ceiling?
req_delta_max = (m_max_target / base_max_m) - 1.0

# Intersect with our allowed Smoothness Limit [-DELTA_LIMIT, +DELTA_LIMIT]
effective_min = max(req_delta_min, -delta_limit)
effective_max = min(req_delta_max, delta_limit)
```
This block calculates the exact range of $\delta$ required to push the star into the target mass window (defined in `CONSTANTS`). If the required $\delta$ is too large (violating smoothness limits), the combination is skipped.

### 2. Physics Setup
Once a valid $\delta$ is sampled, the scaling factor $\alpha$ is computed. A "Transition Jitter" is also applied to the crust-core transition pressure. This adds variability to the low-density region of the EoS, ensuring the ML model does not overfit to specific transition points.

```python
target_m = base_max_m * (1.0 + delta)
alpha = (base_max_m / target_m)**2

# Randomize transition pressure by +/- 20%
p_jitter = np.random.uniform(0.80, 1.20)
p_trans_mix = p_base_mix * p_jitter
```

### 3. TOV Solution and Filters
The `solve_sequence` function is called to generate the M-R curve. Several strict filters are applied immediately after generation:

1.  **Mass Range:** The star must actually fall within the target mass range.
2.  **Early Causality:** The EoS is rejected if the speed of sound saturates ($c_s^2 \approx 1$) at densities too low to be realistic (controlled by `CAUSALITY_EPS_LIMIT`).
3.  **Radius Cap:** Stars with radii $> 14$ km at canonical masses are discarded to match astrophysical constraints.

### 4. Feature Extraction
If the curve is valid, key diagnostic features are extracted via interpolation.

```python
f_R = PchipInterpolator(c_arr[:,0], c_arr[:,1])
f_CS2 = interp1d(c_arr[:,0], c_arr[:,5], kind='linear', ...)

r_14 = float(f_R(1.4))
cs2_at_14 = float(f_CS2(1.4))
```
Slopes are calculated at 1.4, 1.6, 1.8, and 2.0 $M_{\odot}$. If a star is not massive enough to reach these points (e.g., collapses at 1.9 $M_{\odot}$), `NaN` is stored for the missing features.

### 5. Data serialization
The data is flattened into a list format matching `COLUMN_SCHEMA`. Each point on the stable branch is saved as a row in the dataset, labeled with `0` (Hadronic) and a unique `curve_id`.